# TD1 – Kppv et réseaux de neurones pour la classification d'images

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from kppv import *
from functions import *

## Initialisation
On commence par définir une fonction, qui, à partir d'une liste contenant les dimensions des différentes couches du réseau de neurones, initialisera les paramètres.
Pour avoir une initialisation reproductible on fixe np.random.seed(1)

In [2]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)  # number of layers in the network
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * np.sqrt(2/layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        assert (parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert (parameters['b' + str(l)].shape == (layer_dims[l], 1))
    return parameters

## Forward propagation
Ensuite on définit 3 fonctions nécessaires à la propagation en avant. Une fonction **linear_forward** effectuant les produits matriciels de la propagation en avant, puis une fonction **linear_activation_forward**, qui, utilisant **linear_forward** effectue la propagation d'une couche à une autre (produit matriciel + fonction d'activation). On a 3 options de fonctions : relu, sigmoid ou softmax. Par la suite nous utiliserons la fonctions **ReLU** pour les couches cachées, et la fonction **softmax** pour générer les probabilités sur la couche de sortie.
Enfin, **L_model_forward** permet de réaliser la propagation à travers toutes les couches, en utilisant **linear_activation_forward**. C'est le seul endroit où un boucle *for* est inévitable.

In [3]:
def linear_forward(A, W, b):
    std = np.std(A, axis=0, keepdims=True)
    mean = np.mean(A, axis=0, keepdims=True)
    A = (A - mean)/std
    Z = np.dot(W,A)+b
    cache = (A, W, b,std)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network
    for l in range(1, L + 1): # L+1 is pretty important ATTENTION
        A_prev = A
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        if l < L:
            A, cache = linear_activation_forward(A_prev, Wl, bl, activation="relu")
        else:
            # AL, cache = linear_activation_forward(A_prev, Wl, bl, activation="sigmoid")
            AL, cache = linear_activation_forward(A_prev, Wl, bl, activation="softmax")
        caches.append(cache)
    return AL, caches


## Fonction de coût
On définit une fonction de coût. On a J = $\frac{-1}{m}$ $\sum_{i=1}^{m} y_i*log(al_i) $ où AL est le vecteur de sortie obtenue lors de la propagation en avant. C'est la fonction de coût dite de 'cross-entropy'

In [4]:
def compute_cost(AL, Y, activation_out, parameters, lambda_reg):
    m = Y.shape[1]
    L = len(parameters) // 2
    if activation_out == "sigmoid":
        cost = (-1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    elif activation_out == "softmax":
        cost = (-1 / m) * np.sum(Y * np.log(AL))
    reg_cost = 0
    for l in range(1, L):
        reg_cost += np.sum(parameters['W' + str(l)]**2)
    return cost+ lambda_reg*reg_cost/m

## Backward propagation
On définit 3 fonctions nécessaires à la backpropagation. Une fonction **linear_backward** effectuant les produits matriciels de la backpropagation, puis une fonction **linear_activation_backward**, qui, utilisant **linear_backward** propage la backpropagation en fonctione de la fonction d'activation de la couche (utilisation de la fonction backward adéquate).
Les différentes fonctions backward sont définies dans le fichier function.py, afin de ne pas alourdir le propos ici. Elles prennent toutes dA en argument, ainsi que le cache, et retournent dZ.
Enfin, **L_model_backward** permet de réaliser la backpropagation à travers toutes les couches, en utilisant **linear_activation_backward**. C'est le seul endroit où un boucle *for* est inévitable.

In [5]:
def linear_backward(dZ, cache):
    A_prev, W, b, std = cache
    m = A_prev.shape[1]
    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)*std
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "softmax":
        dZ = softmax_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)

    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
    grads['dA' + str(L-1)], grads['dW' + str(L)], grads['db' + str(L)] = linear_activation_backward(dAL, caches[L-1], activation="softmax")  # Softmax or sigmoid, in function of the need
    for l in reversed(range(L-1)):
        grads['dA' + str(l)], grads['dW' + str(l+1)], grads['db' + str(l+1)] = linear_activation_backward(grads['dA' + str(l+1)], caches[l], activation="relu")
    return grads


## Mise à jour des paramètres
Suite à la backpropagation, on définit une fonction pour mettre à jour les paramètres en fonctions des gradients récupérés, des valeurs initiales des paramètres, ainsi que du learning_rate. Ce learning_rate est un hyper-paramètre qu'il nous faudra ajuster par la suite

In [6]:
def adam_grad(grad, beta1, beta2, n_iter): #n_iter is the number of the iteration/ epoch ?
    first_moment = 0
    second_moment = 0
    first_moment = beta1 * first_moment+(1-beta1)*grad
    second_moment = beta2* second_moment + (1-beta2)*grad*grad
    first_unbias = first_moment / (1-beta1**n_iter)
    second_unbias = second_moment / (1-beta2**n_iter)
    return (first_unbias, second_unbias)

def learning_rate_decay(learning_rate_0, n_epoch, N_epoch):
    learning_rate_n = learning_rate_0*(1+np.cos(np.pi*n_epoch/N_epoch))/2  # Commence à 1
    return learning_rate_n


def update_parameters(params, grads, learning_rate, n_epoch, N_epoch, beta1, beta2, learning_r_decay=True, adam=True):
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network
    if learning_r_decay:
        learning_rate_cur = learning_rate_decay(learning_rate, n_epoch, N_epoch)
    else:
        learning_rate_cur = learning_rate
    for l in range(1,L+1):
        if adam:
            first_unbias_w, second_unbias_w = adam_grad(grads['dW' + str(l)], beta1, beta2, n_epoch)
            parameters['W' + str(l)] = parameters['W' + str(l)] -learning_rate*first_unbias_w / (np.sqrt(second_unbias_w)+ 1e-7)
            first_unbias_l, second_unbias_l = adam_grad(grads['db' + str(l)], beta1, beta2, n_epoch)
            parameters['b' + str(l)] = parameters['b' + str(l)] -learning_rate*first_unbias_l / (np.sqrt(second_unbias_l)+ 1e-7)
        else:
            parameters['W' + str(l)] = parameters['W' + str(l)] -learning_rate_cur*grads['dW' + str(l)]
            parameters['b' + str(l)] = parameters['b' + str(l)] -learning_rate_cur*grads['db' + str(l)]
    return parameters

## Modèle complet

In [7]:
# L_layer_model
def L_layer_model(X, Y, parameters, learning_rate, lambda_reg, n_epoch, N_epoch, beta1, beta2, prev_cost,
                  learning_r_dec, adam):

    costn_1 = prev_cost
    AL, caches = L_model_forward(X, parameters)
    cost = compute_cost(AL, Y, 'softmax', parameters, lambda_reg)
    grads = L_model_backward(AL, Y, caches)
    parameters = update_parameters(parameters, grads, learning_rate, n_epoch, N_epoch, beta1, beta2, learning_r_dec, adam)

    if costn_1 < cost:
        print("Error, Alpha parameter to lessen")

    return parameters, cost

## Fonction d'évaluation de la performance

In [8]:
def evaluate_prediction(Ypred, Ytest):
    tot_true = np.sum(Ytest==np.array(np.argmax(Ypred, axis=0)))
    return tot_true/Ytest.size

## Tracé de l'apprentissage

In [9]:
def plot_loss_function(cost):
    n_iter = len(cost)
    iter = np.arange(1, n_iter+1)
    plt.plot(iter, cost)
    plt.title("Cost function")
    plt.show()
    
def plot_accuracy(train_accu, test_accu):
    n_iter = len(train_accu)
    iter = np.arange(1, n_iter+1)
    plt.plot(iter, train_accu, "b--", label="Training accuracy")
    plt.plot(iter, test_accu, "r--", label="Test accuracy")
    plt.legend()
    plt.title("Accuracy")
    plt.show()
    

In [10]:
# Training the model
np.random.seed(1)  # pour que l'exécution soit déterministe

data, labels = read_cifar(short=False)
data = (data - np.mean(data, axis=0))/np.std(data, axis=0)
Xapp, Yapp, Xtest, Ytest = split_data(data, labels)

X_train = Xapp.T
Y_train = np.array([Yapp])
X_test_clean = Xtest.T
Y_test_clean = np.array([Ytest])

def NN_model(X_train, Y_train, X_test_clean, Y_test_clean, N_epoch, m_batch, learning_rate, lambda_reg, 
              beta1=0.9, beta2 = 0.999, learning_r_dec= True, adam=True):
    n_x, m = X_train.shape[0], X_train.shape[1]
    assert m % m_batch ==0
    n_batch = m // m_batch
    L_X_train = np.split(X_train, n_batch, axis=1)
    L_Y_train = np.split(Y_train, n_batch, axis=1)
    uniques = np.unique(Yapp)
    n_y = len(uniques)
    costs = []
    train_accu = []
    test_accu = []
    layers_dims = [n_x, 50, 30, n_y] # 0.0025 good for [50,25]  # Try 0.0001 bcs 0.0005 not working at 1500 it.
    parameters = initialize_parameters_deep(layers_dims)
    
    for k in range(1,N_epoch+1):
        for j in range(n_batch):
            Y_train_final = np.zeros((n_y, L_Y_train[j].shape[1]))
            cost = np.inf
            for i in range(n_y):
                Y_train_final[i,:] = (L_Y_train[j] == uniques[i])
                
            parameters, cost = L_layer_model(L_X_train[j], Y_train_final, parameters, learning_rate, lambda_reg, 
                                   k, N_epoch, beta1, beta2, cost, learning_r_dec, adam)
            costs.append(cost)
            
            pred_train = evaluate_prediction(L_model_forward(L_X_train[j],parameters)[0], Y_train_final)
            pred_test = evaluate_prediction(L_model_forward(X_test_clean,parameters)[0], Y_test_clean)
            train_accu.append(pred_train)
            test_accu.append(pred_test)
            print(cost)
        print(f'NN accuracy on training set \033[1m{pred_train:.3%}\033[0m')
        print(f'NN accuracy on test set \033[1m{pred_test:.3%}\033[0m')
        plot_loss_function(cost)
        plot_accuracy(train_accu, test_accu)

NN_model(X_train, Y_train, X_test_clean, Y_test_clean, N_epoch= 100, m_batch = 128, learning_rate=0.0001, lambda_reg=0,
         beta1=0.9, beta2 = 0.999, learning_r_dec= False, adam=False)

2.9424437430367316
2.9205603391031563
3.14267259243959
2.9354063609786354
2.9930795385596256
2.9365973454518217
3.055950356216418
2.8609757655962285
2.9539781139782537
3.1622771579900437
2.857226299512101
3.0332455874367015
3.0267255477212025
2.8840601402217363
2.8437326395451943
2.7500524690348787
2.718004109607297
3.057639432754966
2.910746533433625
3.1169310775613965
3.0451152899176384
2.8662814894440185
3.039308719023365
2.875299786381128
3.059748910684859
2.9226358915831567
2.92718862622531
3.0169071548697
2.932011280886143
2.8440414686723976
3.1445948739221
2.960871095431054
2.8483752153189013
3.0898720732371263
3.0534091956677
2.74378166134304
2.849680481575449
2.8970465358605346
3.1473388084395326
3.0071607983277904
3.1535713455663497
2.9738768165325187
2.717683584251465
2.707854155953211
2.9831303819463044
2.834741224542766
3.1288981726085314
3.001398803227531
3.0055157674562505
3.007069059515583
2.9483510008758045
3.1043875825815697
3.0518194768862275
2.734702374712584
3.0118

KeyboardInterrupt: 

In [ ]:
data, labels = read_cifar(short=True)
data = (data - np.mean(data, axis=0))/np.std(data, axis=0)
Xapp, Yapp, Xtest, Ytest = split_data(data, labels)

X_train = Xapp.T
Y_train = np.array([Yapp])
X_test_clean = Xtest.T
Y_test_clean = np.array([Ytest])
L_X_train = np.array_split(X_train, n_batch, axis=1)
len(L_X_train)